In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error


In [ ]:
#страшный график корелляции t от x3
import seaborn as sns
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test.csv', nrows = 10000)
sns.countplot(x = 't', hue = 'x3', data = data)

Заполнение t средним значением

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
#попытка заполнить пропущенные значения t с помощью случайного леса
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
for data in pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv', chunksize=10 ** 6):

  # Разделение данных на train_data и missing_data
  train_data = data[~data['t'].isnull() ] # В этой дейте все ок
  missing_data = data[data['t'].isnull()] # А вот эта дейта с пропущенными признаками

  # Разделение train_data на признаки и целевую переменную
  X_train = train_data.drop(['t'], axis=1)
  y_train_x1 = train_data['t']


  # Создание и обучение модели для x1
  model_x1 = RandomForestRegressor()
  model_x1.fit(X_train, y_train_x1)


  # Получение предсказанных значений для missing_data
  X_missing = missing_data.drop(['t'], axis=1)
  missing_data['t'] = model_x1.predict(X_missing)



  # Объединение train_data и missing_data с заполненными значениями
  filled_data = pd.concat([train_data, missing_data])

  # Попытка заполнить средним значением
  mean_t = data['t'].mean()
  filled_data = data.fillna({'t': mean_t})

  filled_data.update(missing_data)

  # Вывод заполненного набора данных
  print(filled_data)
  filled_data.to_csv('filled.csv', index=False)


In [ ]:
import pandas as pd
from sklearn.ensemble import HistGradientBoostingRegressor# реализуем с помощью линейной регрессии



train_data = pd.read_csv('/content/drive/MyDrive/dima/test_etalon40.csv') #данные, на которых учится (объединяем несколько test_etalon в один)
test_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test.csv') #данные для теста
train_data['x1'] = train_data['x1'].fillna(0)
# Разделение на признаки x и целевую переменную (y), 
x_train = train_data[['t', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8']]
y_train = train_data['x1']
x_test = test_data[['t', 'x2', 'x3']]

# Добавление недостающих признаков в тестовых данных

missing_columns = ['x4', 'x5', 'x6', 'x7', 'x8']
for column in missing_columns:
   x_test[column] = 0
model = HistGradientBoostingRegressor()
model.fit(x_train, y_train)

predicted_values = model.predict(x_test)

# Заполнение пропущенных значений в test['x1'] предсказанными значениями
test_data['x1'].fillna(pd.Series(predicted_values), inplace=True)

# Добавление предсказанных значений в тестовый набор
#test_data['predicted_x1'] = predicted_values

# Сохранение тестового набора с предсказанными значениями в файл
test_data
test_data.to_csv('predicted.csv', index=False)


In [ ]:
import pandas as pd
import pickle
from sklearn.ensemble import HistGradientBoostingRegressor# 
from sklearn.ensemble import RandomForestRegressor

filename = '/content/drive/MyDrive/Colab Notebooks/finalized_model_for_fulldata.sav' # Загружаем предыдущий стейт машины

loaded_model = pickle.load(open(filename, 'rb'))

train_data = pd.read_csv('/content/drive/MyDrive/dima1/die6_merged.csv') # Все как обычно -- вводим тренировочные переменные, переменные для предикта

x_train = train_data[['t', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8']]
y_train = train_data['x1']

test_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test.csv')

missing_data = test_data [test_data ['x1'].isnull()] 


x_test = test_data[['t', 'x2', 'x3']]

missing_columns = ['x4', 'x5', 'x6', 'x7', 'x8']  
for column in missing_columns:
    x_test[column] = 1

loaded_model.fit(x_train, y_train) # Загруженная машина предсказывает значения

predicted_values = loaded_model.predict(x_test)


test_data['x1'].fillna(pd.Series(predicted_values), inplace=True)
missing_data ['x1'].fillna(pd.Series(predicted_values), inplace=True)

test_data.to_csv('/content/drive/MyDrive/Colab Notebooks/predicted_values2.csv', index=False) # Дейта со всеми значениями x1

missing_data.to_csv('/content/drive/MyDrive/Colab Notebooks/missing_data.csv', index=False) #Дейта только с пропущенными значениями x1

filename = '/content/drive/MyDrive/Colab Notebooks/finalized_model_for22.sav' # Сохраняем текущее состояние для машины
pickle.dump(loaded_model, open(filename, 'wb'))


In [ ]:
# А это рассчет x4..x8

In [ ]:
import pandas as pd
import networkx as nx

data = pd.read_csv('/content/drive/MyDrive/data/die6.csv',nrows = 1e2)
data

In [ ]:
attr= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/attr.csv')

In [ ]:
ro = attr.value_counts('ego_id')

In [ ]:
import math
import math
import pandas as pd
import networkx as nx
def calc_x4_x7(data,start_index,end_index):
  data_sort = data
  data_sort['x4']=0
  data_sort['x5']=0
  data_sort['x6']=0
  data_sort['x7']=0
  data_sort['x8']=0
  def count_age_koeff(age_diff):
    if age_diff<10:
      return 1
    else:
      return -math.exp(abs(age_diff-10))/math.exp(3)
  for i in range (0,end_index):
    temp = data_sort.loc[i]
    try:
      data_sort.loc[i,'x6']=ro[temp['ego_id']]
    #except:
    #  pass

    #try:
      temp_u = attr[attr['u']==temp['u']][attr['ego_id']==temp['ego_id']]
      temp_v = attr[attr['u']==temp['v']][attr['ego_id']==temp['ego_id']]
      u_age = temp_u.values[0][2]
      v_age = temp_v.values[0][2]
      u_sex = temp_u.values[0][4]
      v_sex = temp_v.values[0][4]
      u_sch = temp_u.values[0][5]
      v_sch = temp_v.values[0][5]
      u_uni = temp_u.values[0][6]
      v_uni = temp_v.values[0][6]
      u_cit = temp_u.values[0][3]
      v_cit = temp_v.values[0][3]
      u_cit = temp_u.values[0][3]
      v_cit = temp_v.values[0][3]
      #print(f"u_sch={u_sch}, v_sch={v_sch}, u_uni = {u_uni},v_uni = {v_uni}")
      temp_x4=count_age_koeff(abs(u_age-v_age))*(0.6*0.0**abs(u_sex-v_sex) + 0.4*0.0**0.0**abs(u_sex-v_sex))
      temp_x5=0.75*0.0**abs(u_sch-v_sch)*0.0**0.0**abs(u_sch+1)*0.0**0.0**abs(v_sch+1)+0.5*0.0**abs(u_uni-v_uni)*0.0**0.0**abs(u_uni+1)*0.0**0.0**abs(v_uni+1)
      temp_x7 = 1 if u_cit-v_cit == 0 else 0.01
      #print(temp_x5)
      data_sort.loc[i,'x4']=temp_x4
      data_sort.loc[i,'x5']=temp_x5
      data_sort.loc[i,'x7']=temp_x7
    except:
      #if i%10==0:
        #print(f"error{i}") 
        #print (temp)
      pass
    data_sort.to_csv(f'/content/drive/MyDrive/dima1_1/test_etalon{start_index}.csv')
#calc_x4_x7(data,0,100)
ROWS=1000
START=0
END=50
for j in range(START,END):
  temp_data = pd.read_csv('/content/drive/MyDrive/data/die6.csv',nrows = ROWS,skiprows=range(1, j*ROWS))
  calc_x4_x7(temp_data,j,ROWS)
